## 결정 트리, 랜덤 포레스트, 에이다부스트. 
#### 필요한 패키지를 불러온다:

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
#from scipy.stats import itemfreq

#### 데이터를 불러온다:

In [26]:
os.chdir(r"D:\python_ML\python_ml_basic\data")

In [27]:
# df = pd.read_csv('data_spam.csv', header='infer',encoding='ISO-8859-1')
df = pd.read_csv('data_spam.csv', header='infer',encoding='latin1')

In [28]:
# 스팸이 없는거 X, 있는거 Y
X=np.array(df.drop(columns='is_spam'))
Y=np.array(df.is_spam)
header = df.columns
headerX = df.drop(columns='is_spam').columns
X[0]

array([3.227e+03, 1.060e+00, 2.600e-01, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 1.850e+00,
       2.600e-01, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 2.600e-01, 0.000e+00, 0.000e+00, 7.700e+01, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.720e+02, 0.000e+00, 0.000e+00,
       0.000e+00, 5.300e-01, 0.000e+00, 2.600e-01, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       2.646e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 3.900e-02])

'yes', 'no' 레이블을 숫자로 변환:

In [29]:
LE = preprocessing.LabelEncoder()
Y = LE.fit_transform(Y)

In [30]:
table = np.unique(Y, return_counts=True)
table

D:\python_ML\venv\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  """Entry point for launching an IPython kernel.


array([[  0, 736],
       [  1, 264]], dtype=int64)

NaN이 있으면 채워 넣음:

In [54]:
IPT = preprocessing.Imputer()
X = IPT.fit_transform(X)

#### 데이터 전처리: (표준화: 데이터 값에 평균 값을 빼고, 표준편차로 나눈다.)

In [63]:
X = preprocessing.scale(X)
X[0]

array([ 0.72081963,  0.57997077,  1.07666106, -0.1484211 , -0.40356631,
       -0.16247803, -0.27369412, -0.25788466, -0.18924522, -0.38261257,
       -0.17878278, -0.35573957,  0.        , -0.06745704, -0.29700331,
       -0.15878435, -0.17192509, -0.12911575,  0.        , -0.07629109,
       -0.04908179, -0.25303435, -0.31342578,  2.24186126,  0.90814171,
       -0.26125725, -0.36128802, -0.16370869, -0.32656806, -0.14422008,
       -0.40541109,  0.45275325, -0.16496962, -0.20187996,  0.28191107,
       -0.15188341, -0.30280737, -0.10393389, -0.22533266, -0.14915475,
       -0.1766068 , -0.58868114, -0.24449336,  0.50935817, -0.91104217,
       -0.04461139,  0.        , -0.17890472, -0.2445704 , -0.16846357,
       -0.32582265, -0.15286473, -0.24685195, -0.25025996, -0.06632812,
       -0.15933516, -0.26808941,  0.        ,  0.        , -0.1111584 ,
       -0.24558099, -0.433543  ])

In [64]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5, random_state=3)

#### Tree 적용 (교차검증 최적화 포함):

In [65]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}

In [75]:
# CV(Cross Validation) 교차검증
gridCV = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV.fit(X_train, Y_train);
best_depth = gridCV.best_params_['max_depth']

In [76]:
print("Tree best depth : " + str(best_depth))

Tree best depth : 10


In [49]:
DTC_best = DecisionTreeClassifier(max_depth=best_depth)
DTC_best.fit(X_train, Y_train);
Y_pred = DTC_best.predict(X_test)
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

Tree best accuracy : 0.89


#### Random Forest 적용 (교차검증 최적화 포함):

Tree 하고는 스텝에서 조금 차이가 있음:

In [ ]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}
gridCV = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_depth = gridCV.best_params_['max_depth']

In [ ]:
print("Random Forest best n estimator : " + str(best_n_estim))
print("Random Forest best depth : " + str(best_depth))

In [ ]:
RF_best = RandomForestClassifier(max_depth=best_depth,n_estimators=best_n_estim,random_state=3)
RF_best.fit(X_train, Y_train);
Y_pred = RF_best.predict(X_test)
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))

#### AdaBoost 적용 (교차검증 최적화 포함): <br>
디폴트: base_estimator=DecisionTreeClassifier 

In [ ]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}
gridCV = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV.fit(X_train, Y_train)
best_n_estim = gridCV.best_params_['n_estimators']
best_learn_rate = gridCV.best_params_['learning_rate']

In [ ]:
print("Ada Boost best n estimator : " + str(best_n_estim))
print("Ada Boost best learning rate : " + str(best_learn_rate))

In [ ]:
AB_best = AdaBoostClassifier(n_estimators=best_n_estim,learning_rate=best_learn_rate,random_state=3)
AB_best.fit(X_train, Y_train);
Y_pred = AB_best.predict(X_test)
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y_test,Y_pred),3)))